# Loading data from S3 to Pandas via Athena

---

---
## Contents

1. [Introduction](#Introduction)
1. [Reading from Redshift](#Reading-from-Redshift)
1. [Upload to S3](#Upload-to-S3)
1. [Writing back to Redshift](#Writing-back-to-Redshift)



## Introduction
In this notebook we illustrate how to copy data from Redshift to S3 and vice-versa.

### Prerequisites
In order to successfully run this notebook, you'll first need to:
1. Have a Redshift cluster within the same VPC.
1. Preload that cluster with data from the [iris data set](https://archive.ics.uci.edu/ml/datasets/iris) in a table named public.irisdata.
1. Update the credential file (`redshift_creds_template.json.nogit`) file with the appropriate information.

Also, note that this Notebook instance needs to resolve to a private IP when connecting to the Redshift instance. There are two ways to resolve the Redshift DNS name to a private IP:
1. The Redshift cluster is not publicly accessible so by default it will resolve to private IP.
1. The Redshift cluster is publicly accessible and has an EIP associated with it but when accessed from within a VPC, it should resolve to private IP of the Redshift cluster. This is possible by setting following two VPC attributes to yes: DNS resolution and DNS hostnames. For instructions on setting that up, see Redshift public docs on [Managing Clusters in an Amazon Virtual Private Cloud (VPC)](https://docs.aws.amazon.com/redshift/latest/mgmt/managing-clusters-vpc.html).

### Notebook Setup
Let's start by installing `PyAthenaJDBC`, a python wrapper for Athena JDBC driver, adding a few imports and specifying a few configs. 

In [4]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.4MB 857kB/s eta 0:00:01
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1


In [5]:
!pip3 install PyAthenaJDBC

  Using cached PyAthenaJDBC-1.3.3-py2.py3-none-any.whl
    100% |████████████████████████████████| 829kB 1.4MB/s eta 0:00:01
  Using cached JPype1-0.6.2.tar.gz
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c2/50/7c/0d83b4baac4f63ff7a765bd16390d2ab43c93587fac9d6017a
  Running setup.py bdist_wheel for jpype1 ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/8e/f3/e6/a1250b8e8d2bd105f4dd21b1dc801dbcf5d815592443bfe741
Successfully built future jpype1


In [23]:
import os
import boto3
import pandas as pd
from pyathenajdbc import connect

region = boto3.Session().region_name

bucket='ar-sm-bucket' # put your s3 bucket name here, and create s3 bucket
prefix = 'sagemaker/DEMO-athena'
# customize to your bucket where you have stored the data

credfile = 'athena_creds.properties'

## Reading from Athena
We store the information needed to connect to Athena in a credentials file. See the file `athena_creds.properties` for an example. 

In [24]:
# Sample query for testing
query = 'select * from elb_logs limit 5;'

We create a connection to athena using our credentials, and use this to query athena and store the result in a pandas DataFrame, which we then save.

In [27]:
print("Reading from Athena...")

def get_conn(credfile): 
    conn = connect(s3_staging_dir='s3://ar-sm-bucket/',credential_file=credfile,schema_name='sampledb', region_name=region)
    return conn

def get_df(credfile, query):
    conn = get_conn(credfile)       
    with conn.cursor() as cur:
        cur.execute(query)
        result_set = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        df = pd.DataFrame.from_records(result_set, columns=colnames)
    return df

df = get_df(credfile, query)

print("Saving file")
localFile = 'elb_logs.csv'
df.to_csv(localFile, index=False)

print("Done")

Reading from Athena...
Saving file
Done


## Upload to S3

In [ ]:
print("Writing to S3...")

fObj = open(localFile, 'rb')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, localFile)).upload_fileobj(fObj)
print("Done")